In [81]:
# LOG IN WITH YOUR LIGO CREDENTIALS
from gwpy.io.kerberos import kinit
kinit(keytab= '../vasileios.skliris.keytab' )
!ligo-proxy-init -k



Your identity: vasileios.skliris@LIGO.ORG

Creating proxy .................................... Done
Your proxy is valid until: Mar 20 04:00:26 2019 GMT


In [82]:
import matplotlib.pyplot as plt
import numpy as np
from gwpy.timeseries import TimeSeries
#from gwpy.plotter import TimeSeriesPlot
from gwpy.segments import DataQualityFlag
from gwpy.segments import Segment
from gwpy.time import to_gps
from gwpy.time import from_gps
from dqsegdb2.query import query_segments
import os


In [83]:
#to_gps('Sep 14 2015 09:50:45.391')

day='Jul 28 2017'

gps_s = to_gps(day) #SEARCH FOR A SPECIFIC DAY
gps_e = gps_s+24*3600     
GPS = [gps_s, gps_e]


H1_ready=query_segments('H1:DMT-ANALYSIS_READY:1', GPS[0],GPS[1])#, url='http://segments.ldas.cit')
L1_ready=query_segments('L1:DMT-ANALYSIS_READY:1', GPS[0],GPS[1])#, url='http://segments.ldas.cit')
V1_ready=query_segments('V1:ITF_SCIENCE:1', GPS[0],GPS[1])#, url='http://segments.ldas.cit')


# PRINT ALL THE SEGMENTS THAT ARE SIMULTANIUS ON 3 DETECTORS

sim_seg = H1_ready['active'] & L1_ready['active'] & V1_ready['active']
if sim_seg==[]:
    print('No simultaneous GPS time, sorry ...')
else:
    seg_time=[]      #DURATION OF EVERY SEGMENT
    sim_time_total=0 #SIMULTANIOUS TIME
    seg_no=0         #NUMBER OF SEGMENTS
    for seg in sim_seg:
        seg_time.append([seg[0],seg[1]])
        sim_time_total+=(seg[1]-seg[0])
        seg_no+=1
    print('There are ',seg_no,' segments of simultaneous GPS time, with total duration of',sim_time_total,' seconds, ',sim_time_total/3600,'hours')
    print('More specifically:')
    for i in range(0,len(sim_seg)):
        print('Segment No:'+str(i)+' Time: '+str(sim_seg[i][0])+' to '+str(sim_seg[i][1])+' Duration: '+str(sim_seg[i][1]-sim_seg[i][0]))    
        
        

There are  3  segments of simultaneous GPS time, with total duration of 24170.0  seconds,  6.713888888888889 hours
More specifically:
Segment No:0 Time: 1185248312.0 to 1185252003.0 Duration: 3691.0
Segment No:1 Time: 1185286802.0 to 1185295603.0 Duration: 8801.0
Segment No:2 Time: 1185295624.0 to 1185307302.0 Duration: 11678.0


In [84]:
#MAKE A DIRECTORY TO STORE THE FILES IF IT DOESN'T ALREADY EXIST 
DATE='20170728'
FS='2048'
file_name=DATE
print(file_name)

os.mkdir('/home/vasileios.skliris/EMILY/ligo_data/'+FS+'/'+file_name)
os.mkdir('/home/vasileios.skliris/EMILY/ligo_data/'+FS+'/'+file_name+'/H')
os.mkdir('/home/vasileios.skliris/EMILY/ligo_data/'+FS+'/'+file_name+'/L')
os.mkdir('/home/vasileios.skliris/EMILY/ligo_data/'+FS+'/'+file_name+'/V')


20170728


In [85]:
downsample=2048  # The data are in 16384 Hz sampling rate, resample to the frequency you want, 2048,4096 ...
segments=[0,1,2] # The segment number of the above that you want to save
fs=2048
saving_type='segments'


for segment in segments:
    
    file_size=32  # Time duration of the time intervals that you will save in grinding

    ####################################################


    gps_start=sim_seg[segment][0]
    gps_end=sim_seg[segment][1]

    datah=TimeSeries.get('H1:GDS-CALIB_STRAIN', gps_start, gps_end, frametype='H1_HOFT_C00',verbose=True)
    datal=TimeSeries.get('L1:GDS-CALIB_STRAIN', gps_start, gps_end, frametype='L1_HOFT_C00',verbose=True)
    datav=TimeSeries.get('V1:Hrec_hoft_16384Hz', gps_start, gps_end, frametype='V1Online',verbose=True)


    if isinstance(downsample,int):
        datah=datah.resample(downsample)
        datal=datal.resample(downsample)
        datav=datav.resample(downsample)

    if saving_type=='grinding':

        no_of_sets=int((sim_seg[segment][1]-sim_seg[segment][0])/file_size)

        for i in range(0,no_of_sets-1):
            start=int(i*file_size)
            end=int((i+1)*file_size)

            h=datah[start*fs:end*fs]
            l=datal[start*fs:end*fs]
            v=datav[start*fs:end*fs]
            #Saving as many segments of file_size time intervals from a specific segment


            nameref=str(segment)+'_'+str(int(gps_start+start))+'.txt'

            np.savetxt('/home/vasileios.skliris/EMILY/ligo_data/'+FS+'/'+file_name+'/H/'+nameref,h)
            print(str(i)+'. File '+nameref+' is successfully saved')
            np.savetxt('/home/vasileios.skliris/EMILY/ligo_data/'+FS+'/'+file_name+'/L/'+nameref,l)
            print('   File '+nameref+' is successfully saved')
            np.savetxt('/home/vasileios.skliris/EMILY/ligo_data/'+FS+'/'+file_name+'/V/'+nameref,v)
            print('   File '+nameref+' is successfully saved')


    elif saving_type=='segments':


        nameref='SEG'+str(segment)+'_'+str(int(gps_start))+'_'+str(int(gps_end-gps_start))+'s.txt'

        np.savetxt('/home/vasileios.skliris/EMILY/ligo_data/'+FS+'/'+file_name+'/H/'+nameref,datah)
        print('File '+nameref+' is successfully saved')
        np.savetxt('/home/vasileios.skliris/EMILY/ligo_data/'+FS+'/'+file_name+'/L/'+nameref,datal)
        print('File '+nameref+' is successfully saved')
        np.savetxt('/home/vasileios.skliris/EMILY/ligo_data/'+FS+'/'+file_name+'/V/'+nameref,datav)
        print('File '+nameref+' is successfully saved')

Attempting to access data from frames...

Reading H1_HOFT_C00 frames: |          | 0/2 (  0%) ETA ?     
Reading H1_HOFT_C00 frames: |          | 0/2 (  0%) ETA ?     

no GWF API available, please install a third-party GWF library (framecpp, lalframe) and try again


Failed to access data from frames, trying NDS...

Reading H1_HOFT_C00 frames: |█████     | 1/2 ( 50%) ETA 00:00 Opening new connection to 10.21.2.4... connected
    [10.21.2.4] set ALLOW_DATA_ON_TAPE='False'
Checking channels list against NDS2 database... done


Reading L1_HOFT_C00 frames: |          | 0/2 (  0%) ETA ?     
Reading L1_HOFT_C00 frames: |          | 0/2 (  0%) ETA ?     

no GWF API available, please install a third-party GWF library (framecpp, lalframe) and try again


Failed to access data from frames, trying NDS...

Reading L1_HOFT_C00 frames: |█████     | 1/2 ( 50%) ETA 00:00 Opening new connection to 10.21.2.4... connected
    [10.21.2.4] set ALLOW_DATA_ON_TAPE='False'
Checking channels list against NDS2 database... Opening new connection to nds.ligo.caltech.edu... 

...

Failed to access data from frames, trying NDS...

Reading L1_HOFT_C00 frames: |██▌       | 1/4 ( 25%) ETA 00:00 Opening new connection to 10.21.2.4... connected
    [10.21.2.4] set ALLOW_DATA_ON_TAPE='False'
Checking channels list against NDS2 database... Opening new connection to nds.ligo.caltech.edu... 

/home/vasileios.skliris/.conda/envs/conda_env/lib/python3.6/site-packages/gwpy/timeseries/core.py:1114: NDSWarning: unique NDS2 channel match not found for 'L1:GDS-CALIB_STRAIN'
  io_nds2.NDSWarning)


connected
    [nds.ligo.caltech.edu] set ALLOW_DATA_ON_TAPE='False'
Checking channels list against NDS2 database... done


Reading V1Online frames: |          | 0/7 (  0%) ETA ?     
Reading V1Online frames: |          | 0/7 (  0%) ETA ?     

no GWF API available, please install a third-party GWF library (framecpp, lalframe) and try again


Failed to access data from frames, trying NDS...

Reading V1Online frames: |█▍        | 1/7 ( 14%) ETA 00:00 Opening new connection to 10.21.2.4... connected
    [10.21.2.4] set ALLOW_DATA_ON_TAPE='False'
Checking channels list against NDS2 database... done

File SEG2_1185295624_11678s.txt is successfully saved
File SEG2_1185295624_11678s.txt is successfully saved
